<a href="https://colab.research.google.com/github/akash02ita/CPSC-599.6-DL-for-Vision/blob/main/a2/DL4Vision_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2
In this assignment, we will use a multi-layer perceptron network to build an image classifier for single digits. We will be using a public dataset for model development. The dataset we will be using is the MNIST digit dataset. The dataset contains 10 classes, where class `i` contains images of digit `i`.

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import torch.optim as optim

####1. Create train_data and test_dataset objects using the MNIST digit dataset from torchvision.datasets module. (5 points)

In [2]:
transform = transforms.Compose([transforms.ToTensor()])
train_data = datasets.MNIST(root='data', train = True, transform=transform, download = True)
test_dataset = datasets.MNIST(root='data', train = False, transform=transform, download = True)

####2. Use the [random_split](https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split) method to split the `train_data` into `train_dataset` (50000 images) and `validation_dataset` dataset (10000 images). (5 points)

In [3]:
print(len(train_data))
train_dataset, validation_dataset = random_split(train_data, [50000, 10000], generator=torch.Generator().manual_seed(42))
len(train_dataset), len(validation_dataset)

60000


(50000, 10000)

#### 3. Create dataloader objects for `train_dataset`, `validation_dataset`, and `test_dataset`. (5 points)

In [4]:
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 32, shuffle = True)
validation_dataloader = DataLoader(validation_dataset, batch_size = 32, shuffle = True)
type(train_dataloader), len(train_dataloader), len(train_dataloader.dataset)

(torch.utils.data.dataloader.DataLoader, 1563, 50000)

In [5]:
for images, labels in train_dataloader:
  print(type(images), type(labels))
  print(images.shape, labels.shape)
  print(images[0].numpy().shape)
  break

<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([32, 1, 28, 28]) torch.Size([32])
(1, 28, 28)


#### 4. Develop an MLP model for classifying MNIST images. The developed model should have four hidden layers of 256, 128, 64, and 32 neurons. Each hidden layer should be followed with a ReLU unit and a Dropout layer (p=0.2).  (15 points)

In [6]:
def initialize_model(flattened_image_shape):
  model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(flattened_image_shape, 256, bias=True),
    nn.ReLU(),
    nn.Linear(256, 128, bias=True),
    nn.ReLU(),
    nn.Linear(128, 64, bias=True),
    nn.ReLU(),
    nn.Linear(64, 32, bias=True),
    nn.Dropout(p=0.2),
    nn.Linear(32, 10, bias=True)
  )
  return model

flattened_image_shape = 1*28*28
model = initialize_model(flattened_image_shape)

#### 5. Define the components needed for training a deep learning model. (10 points)



In [7]:
def initialize_componenets(model):
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(device)
  model = model.to(device) # returns model on the device

  return criterion, optimizer, device, model

criterion, optimizer, device, model = initialize_componenets(model)

cuda


#### 6. Write the training loop and train the model for 100 epochs. Print the training and validation accuracy and loss for each epoch. (35 points)

In [8]:
def train_model(model, epochs, train_dataloader, validation_dataloader, criterion, optimizer):
  model.train()
  for epoch in range(epochs):
    for current_data, current_dataloader  in [("TRAIN", train_dataloader), ("VALIDATION", validation_dataloader)]:
      total_loss = 0
      total = 0
      correct = 0
      for images, labels in current_dataloader:
        optimizer.zero_grad()
        # data also needs to move to same device (in this case gpu)
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        if current_data == "TRAIN": # only when using training dataloader
          loss.backward()
          optimizer.step()
        total_loss += loss.item() * images.size(0)
        total += images.size(0)
        _, preds = torch.max(outputs, 1) # preds is index of maximum found in each vector ouput \element of ouputs
        correct += (preds == labels).sum().item()
      accuracy = correct / total
      loss = total_loss / total
      print(f"{epoch} {current_data}: Accuracy = {accuracy}, Loss = {loss}")

epochs = 100
train_model(model, epochs, train_dataloader, validation_dataloader, criterion, optimizer)

0 TRAIN: Accuracy = 0.7834, Loss = 0.6471620740687847
0 VALIDATION: Accuracy = 0.9465, Loss = 0.189068625664711
1 TRAIN: Accuracy = 0.95664, Loss = 0.1465376280272007
1 VALIDATION: Accuracy = 0.9534, Loss = 0.16041980398595332
2 TRAIN: Accuracy = 0.97056, Loss = 0.10012527960941195
2 VALIDATION: Accuracy = 0.9665, Loss = 0.11403801368251443
3 TRAIN: Accuracy = 0.97746, Loss = 0.07510992875277996
3 VALIDATION: Accuracy = 0.9679, Loss = 0.10538392783850431
4 TRAIN: Accuracy = 0.98164, Loss = 0.060586594700552526
4 VALIDATION: Accuracy = 0.9703, Loss = 0.11021530694328248
5 TRAIN: Accuracy = 0.98542, Loss = 0.04686100335727446
5 VALIDATION: Accuracy = 0.9722, Loss = 0.09911708828080446
6 TRAIN: Accuracy = 0.98854, Loss = 0.03653769023457076
6 VALIDATION: Accuracy = 0.9729, Loss = 0.09923934811912477
7 TRAIN: Accuracy = 0.98998, Loss = 0.03250537044703262
7 VALIDATION: Accuracy = 0.9694, Loss = 0.12057777505242266
8 TRAIN: Accuracy = 0.99084, Loss = 0.028516112622206564
8 VALIDATION: Accur

####6. Test the model using the `test_dataset`, and report accuracy and loss. (10 points)

In [9]:
def evaluate_model(model, test_dataloader):
  model.eval()
  total_loss = 0
  total = 0
  correct = 0
  for images, labels in test_dataloader:
    images, labels = images.to(device), labels.to(device)
    with torch.no_grad():
      outputs = model(images)
      loss = criterion(outputs, labels)
      total_loss += loss.item() * images.size(0)
      total += images.size(0)
      _, preds = torch.max(outputs, 1) # preds is index of maximum found in each vector ouput \element of ouputs
      correct += (preds == labels).sum().item()
  accuracy = correct / total
  loss = total_loss / total
  print(f"TEST: Accuracy = {accuracy}, Loss = {loss}")

evaluate_model(model, test_dataloader)

TEST: Accuracy = 0.9835, Loss = 0.14146355858886092


#### 8. We would like to see how accurate the trained model is when applied to a set of images of digits with slight differences. The image of a digit in the MNIST dataset has a black background (0 value for pixel values). This might be like writing with white chalk on a blackboard. We make slight changes in the test datasets by applying a simple change `image = 1 - image`. In other words, we invert the pixel intensity values. The resulting images resemble a digit written with a black marker on a whiteboard. Test the model using the updated test_dataset, and report your observations regarding model performance. How would you change your pipeline if you redo this experiment? (15 points)

In [11]:
new_transform = transforms.Compose([transforms.ToTensor(), lambda image : 1-image])
new_test_dataset = datasets.MNIST(root='data', train = False, transform=new_transform, download = True)
new_test_dataloader = DataLoader(new_test_dataset, batch_size = 32, shuffle = True)
evaluate_model(model, new_test_dataloader)

TEST: Accuracy = 0.0688, Loss = 17.06925329284668


**8: observations**

Observing the result above the accuracy of roughly 7% is not good, when images are inverted. The model returns accurate values only for non-inverted images.

**8: how would i change my pipeline to re-do the experiment?**

The model was not trained to handle inverted images. Therefore, image augmentation can come into a play during the training phase.
While passing always the inverted image to the training model most likely causes the model to have low accuracy the moment it is tested and evaluated with non-inverted images, occasionally augmenting or not-augmenting an image in a probabilistic manner is the idea.

Therefore, the **transform** parameter when creating `train_dataset` is a composition of function which will include an additional function. That such additional function will either return the same image or invert the image on a probability basis.

In [13]:
from random import random
transform = transforms.Compose([transforms.ToTensor(), lambda image: 1-image if random() < 0.5 else image]) # image is augmented or original half of the times each
train_data = datasets.MNIST(root='data', train = True, transform=transform, download = True)
test_dataset = datasets.MNIST(root='data', train = False, transform=transform, download = True)
train_dataset, validation_dataset = random_split(train_data, [50000, 10000], generator=torch.Generator().manual_seed(42))

train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 32, shuffle = True)
validation_dataloader = DataLoader(validation_dataset, batch_size = 32, shuffle = True)

flattened_image_shape = 1*28*28
model = initialize_model(flattened_image_shape)
criterion, optimizer, device, model = initialize_componenets(model)

print("---------------------------MODEL GOING TO BE TRAINED")
epochs = 100
train_model(model, epochs, train_dataloader, validation_dataloader, criterion, optimizer)

print("\n---------------------------MODEL GOING TO BE TESTED")
evaluate_model(model, test_dataloader) # model tests with a mixture of inverted and non-inverted images

cuda
---------------------------MODEL GOING TO BE TRAINED
0 TRAIN: Accuracy = 0.62674, Loss = 1.0688356064987183
0 VALIDATION: Accuracy = 0.861, Loss = 0.4586121523618698
1 TRAIN: Accuracy = 0.88878, Loss = 0.3634404557991028
1 VALIDATION: Accuracy = 0.9081, Loss = 0.3169462372541428
2 TRAIN: Accuracy = 0.91922, Loss = 0.27005727748274805
2 VALIDATION: Accuracy = 0.9153, Loss = 0.30177380818128585
3 TRAIN: Accuracy = 0.9331, Loss = 0.22273350422680377
3 VALIDATION: Accuracy = 0.936, Loss = 0.20992755261063575
4 TRAIN: Accuracy = 0.94322, Loss = 0.18765878173619507
4 VALIDATION: Accuracy = 0.9453, Loss = 0.1864306109651923
5 TRAIN: Accuracy = 0.94774, Loss = 0.17288389999091625
5 VALIDATION: Accuracy = 0.9394, Loss = 0.2044623752504587
6 TRAIN: Accuracy = 0.95272, Loss = 0.15720064076691867
6 VALIDATION: Accuracy = 0.9561, Loss = 0.14767234534919263
7 TRAIN: Accuracy = 0.95642, Loss = 0.1422539631064236
7 VALIDATION: Accuracy = 0.9464, Loss = 0.18599360574781895
8 TRAIN: Accuracy = 0.95

In [14]:
print("testing model WITHOUT inverting images")
new_transform = transforms.Compose([transforms.ToTensor()]) # ALL images are NOT inverted
new_test_dataset = datasets.MNIST(root='data', train = False, transform=new_transform, download = True)
new_test_dataloader = DataLoader(new_test_dataset, batch_size = 32, shuffle = True)
evaluate_model(model, new_test_dataloader)

print()

print("testing model WITH inverted images")
new_transform = transforms.Compose([transforms.ToTensor(), lambda image : 1-image]) # ALL images will be inverted
new_test_dataset = datasets.MNIST(root='data', train = False, transform=new_transform, download = True)
new_test_dataloader = DataLoader(new_test_dataset, batch_size = 32, shuffle = True)
evaluate_model(model, new_test_dataloader)

testing model WITHOUT inverting images
TEST: Accuracy = 0.9813, Loss = 0.13317162412837838

testing model WITH inverted images
TEST: Accuracy = 0.9691, Loss = 0.15458672419657524


As shown above, the model accuracy now is satisfiable both for **inverted images** and **non-inverted images**.